# Import Assignments From a CSV File¶
In this example, a CSV file containing the locations of potholes will be imported into a Workforce Project as new assignments.

In [2]:

import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce
from arcgis.geocoding import geocode


### Connect to Organization And Get The Project
Let's connect to ArcGIS Online and find the new Project to add assignments to.

In [3]:

gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)

Enter password: ········



### Load the CSV File¶
Let's use the pandas library to read the CSV file and display the potholes.

In [9]:

df = pd.read_csv("assignments.csv")
df

,xField,yField,Type,Location,Dispatcher,Description,Priority,Work Order Id,Due Date,Attachment,Worker
0,-116.539453,33.829361,Gardening,480 E Tahquitz Canyon Way Palm Springs,workforce_script,Need to remove the fallen tree,high,1,3/13/2018 22:43:00,NaN,aaron_nitro
1,-116.539490,33.831087,Gardening,456 E Andreas Rd Palm Springs,workforce_script,Need to plant a new tree here,medium,2,3/13/2018 21:43:05,NaN,aaron_nitro
2,-116.541971,33.827400,Tree Removal,200 E Arenas Rd Palm Springs,workforce_script,Clean up the trash in the street,low,3,3/13/2018 19:43:09,../uc_data/attachments/trash.png,aaron_nitro
3,-116.546015,33.831499,Gardening,Skyline Trailhead Palm Springs,workforce_script,Clean up the trash near the trailhead,critical,4,3/14/2018 10:43:00,../uc_data/attachments/trash.png,NaN
4,-116.535492,33.840141,Gardening,Ruth Hardy Park Palm Springs,workforce_script,The gardens need to be weeded,high,5,3/14/2018 18:43:09,NaN,NaN
5,-116.524686,33.830887,Gardening,196 N Sunrise Way Palm Springs,workforce_script,Clean up trash,low,6,3/15/2018 14:00:00,NaN,NaN


### Create An Assignment For Each Row
For each assignment, First geocode the address to get the x,y location in (WGS84 Web Mercator) of the assignment. Then supply additional attributes.

Finally use the batch_add method to add multiple assignments at once (this is faster than using the add method since validation is performed once for all assignments).

In [10]:
assignments = []
for index, row in df.iterrows():
    geometry = geocode(f"{row['Location']}", out_sr=3857)[0]["location"]
    assignments.append(
        workforce.Assignment(
            project,
            geometry=geometry,
            location=row["Location"],
            description=row["Description"],
            priority=row["Priority"],
            work_order_id=row["Work Order Id"],
            assignment_type="Fill in Pothole",
            status="unassigned"
        )
    )
project.assignments.batch_add(assignments)

[<Assignment 129>,
 <Assignment 130>,
 <Assignment 131>,
 <Assignment 132>,
 <Assignment 133>,
 <Assignment 134>]


### Verify the assignments on the map
Let's verify that the assignments were created.

In [11]:
webmap = gis.map("Palm Springs", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap

MapView(layout=Layout(height='400px', width='100%'), zoom=14.0)